# 1. Importing Packages

In [1]:
import os
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

In [2]:

#Upload The Following Files
from unet import build_unet
from customMetrics import dice_loss, dice_coef, iou, jaccard_coef, jaccard_loss

# 2. Setting The Environment

In [3]:
H = 128
W = 128

In [4]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Hyperparameters """
batch_size = 32

lr = 1e-4
num_epochs = 100
model_path = os.path.join("/content/files128", "model128_basicJac.h5")
csv_path = os.path.join("/content/files128", "data_basicJac.csv")

In [5]:
""" Dataset """
# Make The Folder and upload the data
dataset_path = "/content/drive/MyDrive/gamma_data_128"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "valid")

In [6]:
!rm -rf files128

In [7]:
!rm -rf logs

In [8]:
pwd

'/content'

# 3. Utils Function

In [9]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [10]:
def load_data(path):
    print(path)
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "gt", "*.jpg")))
    return x, y

In [11]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [12]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

In [13]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [14]:
def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [15]:
train_x, train_y = load_data(train_path)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

/content/drive/MyDrive/gamma_data_128/train
/content/drive/MyDrive/gamma_data_128/valid
Train: 1890 - 1890
Valid: 630 - 630


In [16]:
pwd

'/content'

In [17]:
create_dir("/content/files128")

In [18]:
train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_steps += 1

In [19]:
train_steps, valid_steps

(60, 20)

# 4. The Model

In [20]:
""" Model """
model = build_unet((H, W, 3))
model.compile(loss= jaccard_loss, optimizer=Adam(lr), metrics=[jaccard_coef, dice_coef, iou, Recall(), Precision()])
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________________

In [21]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor= "val_jaccard_coef", factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor= "val_jaccard_coef", patience=10, restore_best_weights=False)
]

In [22]:
   model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks
)

Epoch 1/100
60/60 [==============================] - 1158s 19s/step - loss: -0.2201 - jaccard_coef: 0.2222 - dice_coef: 0.3576 - iou: 0.2222 - recall: 0.6604 - precision: 0.3742 - val_loss: -0.0700 - val_jaccard_coef: 0.0700 - val_dice_coef: 0.1307 - val_iou: 0.0700 - val_recall: 5.5500e-04 - val_precision: 0.0160

Epoch 00001: val_loss improved from inf to -0.06998, saving model to /content/files128/model128_basicJac.h5
Epoch 2/100
60/60 [==============================] - 17s 283ms/step - loss: -0.3638 - jaccard_coef: 0.3652 - dice_coef: 0.5331 - iou: 0.3652 - recall: 0.4714 - precision: 0.7198 - val_loss: -0.0584 - val_jaccard_coef: 0.0584 - val_dice_coef: 0.1103 - val_iou: 0.0584 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00

Epoch 00002: val_loss did not improve from -0.06998
Epoch 3/100
60/60 [==============================] - 17s 284ms/step - loss: -0.4165 - jaccard_coef: 0.4180 - dice_coef: 0.5879 - iou: 0.4180 - recall: 0.4669 - precision: 0.7669 - val_loss: -0.0465 - va